## Additional Information About Hardware:

In [2]:
!nvidia-smi

NVIDIA-SMI has failed because it couldn't communicate with the NVIDIA driver. Make sure that the latest NVIDIA driver is installed and running.



In [3]:
from tensorflow.python.client import  device_lib
device_lib.list_local_devices()

[name: "/device:CPU:0"
 device_type: "CPU"
 memory_limit: 268435456
 locality {
 }
 incarnation: 16753271635956218152]

In [4]:
!cat /proc/cpuinfo

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

In [5]:
!cat /proc/meminfo

MemTotal:       13333580 kB
MemFree:        10785084 kB
MemAvailable:   12450572 kB
Buffers:           74800 kB
Cached:          1731076 kB
SwapCached:            0 kB
Active:           718544 kB
Inactive:        1580916 kB
Active(anon):     452388 kB
Inactive(anon):      332 kB
Active(file):     266156 kB
Inactive(file):  1580584 kB
Unevictable:           0 kB
Mlocked:               0 kB
SwapTotal:             0 kB
SwapFree:              0 kB
Dirty:               556 kB
Writeback:             0 kB
AnonPages:        493412 kB
Mapped:           330564 kB
Shmem:               936 kB
Slab:             157536 kB
SReclaimable:     119328 kB
SUnreclaim:        38208 kB
KernelStack:        3712 kB
PageTables:         6292 kB
NFS_Unstable:          0 kB
Bounce:                0 kB
WritebackTmp:          0 kB
CommitLimit:     6666788 kB
Committed_AS:    2799320 kB
VmallocTotal:   34359738367 kB
VmallocUsed:           0 kB
VmallocChunk:          0 kB
Percpu:              952 kB
AnonHugePages:   

## Mounting Google Drive:

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [8]:
os.chdir('/content/drive/MyDrive/dogCatClassifier') # Only un-comment if rumming in google colab

In [9]:
data_path_train = os.path.join(os.getcwd(), 'datasets', 'extracted_data', 'train') 

In [11]:
# Data Preprocessing:

#1. Define image preprocessor for train
img_train_read = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,
                                   brightness_range = [0.7, 1.0],
                                   rotation_range = 20,
                              )

# Training Dataset:

train_generator = img_train_read.flow_from_directory(
    data_path_train,
    target_size = (150, 150),
   color_mode = 'rgb',
   batch_size = 64,
   class_mode = 'binary',
   shuffle = True) # set as training data

Found 25001 images belonging to 2 classes.


In [12]:
cnn = tf.keras.models.Sequential()


# Adding Layers.
cnn.add(Conv2D(filters=32, kernel_size=3, padding='same', input_shape=[150, 150, 3], activation='relu'))  # kernel_initializer not required

# Step 2 - Pooling
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Adding a second convolutional layer
cnn.add(tf.keras.layers.Conv2D(filters=32, padding='same', kernel_size=3, activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2))

# Step 3 - Flattening
cnn.add(tf.keras.layers.Flatten())

# Step 4 - Full Connection
cnn.add(tf.keras.layers.Dense(units=256, activation='relu'))

# Step 5 - Output Layer
cnn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

In [13]:
# Model Checkpointing for saving model every epochs

filepath_chpt = os.path.join(os.getcwd(), 'tmp', 'fianlChpt', 'final-epoch-{epoch:02d}-acc-{accuracy:.5f}.hdf5')

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
                                  filepath=filepath_chpt,
                                   monitor='val_loss',
                                   verbose=0,
                                   save_freq='epoch',
                                   save_best_only=False,
                                   save_weights_only=False,
                                   mode='auto')

In [14]:
cnn.compile(optimizer = 'adam',
            loss = 'binary_crossentropy',
            metrics = ['accuracy'])

In [15]:
train_chart = pd.read_csv("epoch_accuracy_data.csv")
train_chart

,loss,accuracy,val_loss,val_accuracy
0,0.659772,0.633168,0.553994,0.7228
1,0.556409,0.712814,0.532322,0.7288
2,0.513469,0.748163,0.498645,0.7582
3,0.484697,0.765062,0.476762,0.7734
4,0.472511,0.771911,0.470020,0.7708
5,0.448501,0.788511,0.445666,0.8028
6,0.435729,0.796610,0.445401,0.7992
7,0.417044,0.807210,0.414341,0.8052
8,0.406995,0.812609,0.436279,0.7932
9,0.393792,0.822359,0.407620,0.8136


**15th index (16th epoch) is the best this far....**

**So finally lets train the model for 16 epochs only**

In [16]:
# Training the CNN on the Training set and evaluating it on the Test set
cnn.fit(x = train_generator,
        epochs = 16,
        batch_size = 64,
        callbacks=[model_checkpoint_callback])

Epoch 1/16
391/391 [==============================] - 6057s 15s/step - loss: 0.7759 - accuracy: 0.5776
Epoch 2/16
391/391 [==============================] - 684s 2s/step - loss: 0.5707 - accuracy: 0.7017
Epoch 3/16
391/391 [==============================] - 671s 2s/step - loss: 0.5111 - accuracy: 0.7481
Epoch 4/16
391/391 [==============================] - 668s 2s/step - loss: 0.4732 - accuracy: 0.7739
Epoch 5/16
391/391 [==============================] - 665s 2s/step - loss: 0.4614 - accuracy: 0.7777
Epoch 6/16
391/391 [==============================] - 668s 2s/step - loss: 0.4344 - accuracy: 0.7965
Epoch 7/16
391/391 [==============================] - 668s 2s/step - loss: 0.4272 - accuracy: 0.8013
Epoch 8/16
391/391 [==============================] - 668s 2s/step - loss: 0.4147 - accuracy: 0.8078
Epoch 9/16
391/391 [==============================] - 678s 2s/step - loss: 0.3935 - accuracy: 0.8169
Epoch 10/16
391/391 [==============================] - 780s 2s/step - loss: 0.3896 - accu

In [1]:
history = _
history.params

AttributeError: ignored

In [ ]:
chart = pd.DataFrame(history.history)
chart

In [ ]:
pd.DataFrame(chart).plot(figsize=(8, 5))    # Width, Height
plt.grid(True)
plt.gca().set_ylim(0, 1)
plt.show()

In [17]:
cnn.save('saved_models/final_model/CNN_full_epochs_{epoch:02d}.h5')